You can follow along and play with this notebook by clicking the badge below

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jasongfleischer/COGS118A/blob/main/demo_notebooks/lecture_19_ensembles.ipynb)

# Bagging

Each learner in an ensemble must 
1. be at least a bit better than random chance at the job
2. make mistakes independently from the other learners

A classic way to enforce condition (2) is to make each learner train on a resampled version of the training data; different training data leads to different answers!  

Bootstrap samples can be used to construct an ensemble of any estimator in sklearn through `BaggingClassifier()` or `BaggingRegressor()`.  Just pass it the desired estimator and details about how to construct the bootstrap.  The output of the ensemble will be the median of the outputs of the learners.

```python 
class sklearn.ensemble.BaggingClassifier(estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0, base_estimator='deprecated')
```


In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_hastie_10_2

# dataset is 10-d random uniform, with postive class
# being points > a quadratic surface in that vector space
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=0.5)
bagging.fit(X_train, y_train)
bagging.score(X_test, y_test)

0.7359

Remember that a decision tree looks at every single feature and every single possible threshold for that feature... it chooses the feature that gives the most purity in the resulting split (max information gain), then keeps splitting that way. 
```python
class sklearn.tree.DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0)
```

A random forest is a decision tree based bagging classifier with an extra innovation to force condition (2)... the features get subsampled as well as the datapoints.

```python
class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
```
 

Extra-randomized trees are like random forests with extra randomness in the construction of the tree. A random subset of candidate features is used, but instead of looking for the most discriminative thresholds, thresholds are drawn at random for each candidate feature and the best of these randomly-generated thresholds is picked as the splitting rule. This reduces the variance of the model a bit more, at the expense of a slightly greater increase in bias:
```python
class sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
```

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0).fit(X_train, y_train)
forest = RandomForestClassifier(n_estimators=500, max_depth=None,
    min_samples_split=2, random_state=0).fit(X_train, y_train)
extra = ExtraTreesClassifier(n_estimators=500, max_depth=None,
    min_samples_split=2, random_state=0).fit(X_train, y_train)

print( tree.score(X_test,y_test), forest.score(X_test,y_test), 
      extra.score(X_test,y_test) )

0.7272 0.8766 0.8998


There are regressor versions of all the classifiers above.

# Boosting

The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction. The data modifications at each so-called boosting iteration consist of applying weights $w_1, w_2, w_n$ to each of the training samples. Initially, those weights are all set to $w_i = 1/n$, so that the first step simply trains a weak learner on the original data. For each successive iteration, the sample weights are individually modified and the learning algorithm is reapplied to the reweighted data. At a given step, those training examples that were incorrectly predicted by the boosted model induced at the previous step have their weights increased, whereas the weights are decreased for those that were predicted correctly. As iterations proceed, examples that are difficult to predict receive ever-increasing influence. Each subsequent weak learner is thereby forced to concentrate on the examples that are missed by the previous ones in the sequence

In [10]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=500).fit(X_train,y_train)
ada.score(X_test,y_test)

0.9442

Gradient Tree Boosting or Gradient Boosted Decision Trees (GBDT) is a generalization of boosting to arbitrary differentiable loss functions.

There are two versions of the Gradient methods in sklearn. The first (exact) method is to actually take the full dataset and calculate the gradient on it.  

```python
class sklearn.ensemble.GradientBoostingClassifier(*, loss='log_loss', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
```

The second method is ORDERS OF MAGNITUDE faster for large N datasets. These fast estimators first bin the input samples X into integer-valued bins (typically 256 bins) which tremendously reduces the number of splitting points to consider, and allows the algorithm to leverage integer-based data structures (histograms) instead of relying on sorted continuous values when building the trees. The API of these estimators is slightly different, and some of the features from GradientBoostingClassifier are not yet supported.

```python
class sklearn.ensemble.HistGradientBoostingClassifier(loss='log_loss', *, learning_rate=0.1, max_iter=100, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20, l2_regularization=0.0, max_bins=255, categorical_features=None, monotonic_cst=None, interaction_cst=None, warm_start=False, early_stopping='auto', scoring='loss', validation_fraction=0.1, n_iter_no_change=10, tol=1e-07, verbose=0, random_state=None, class_weight=None)
```

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# high learning rate enabled by the clean 
# quadratic nature of the problem
grad = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

# dunno WTF but its max_iter instead of n_estimators
# to control how many learners are made
hgrad = HistGradientBoostingClassifier(max_iter=500, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

print(grad.score(X_test, y_test), hgrad.score(X_test, y_test))

0.9452 0.9439
